In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from typing import Any # import Any from the typing module

# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

def initialize_driver():
    # Initialize the Chrome driver
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    # Load the page
    driver.get(url)
    time.sleep(5) # wait for the page to load

# Functions to scrape bus routes
def scrape_bus_routes(driver):
    # Scrape bus routes
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Functions to scrape bus details
def scrape_bus_details(driver, url, route_name):
    # Scrape bus details
    try:
        driver.get(url)
        time.sleep(5) # wait for some 5 sec for the page to load

        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5) # wait for some 5 sec for the page to load

            #scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5) # wait for some 5 sec for the page to load

            # Find but details from the availbility
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPATH to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text.strip(),
                    "Bus_Type": bus_type_elements[i].text.strip(),
                    "Departing_Time": departing_time_elements[i].text.strip(),
                    "Duration": duration_elements[i].text.strip(),
                    "Reaching_Time": reaching_time_elements[i].text.strip(),
                    "Star_Rating": star_rating_elements[i].text.strip() if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text.strip(),
                    "Seat_Availability": seat_availability_elements[i].text.strip() if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
           print(f"Error occurred while scrapping bus details for {url}: {str(e)}")
           return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []
# List to hold all bus details
all_bus_details = []

# Main function to execute the scraping process
def scrap_all_pages():
    for page in range(1, 5): # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)

            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5) # wait for some 5 sec for the page to load
                
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)

            # Iterate over each bus route link and scrape details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                  all_bus_details.extend(bus_details)

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrap_all_pages()

# Create a DataFrame from the scraped data
df_buses_6 = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df_buses_6.to_csv('Kadamba_bus_details.csv', index=False)

Error occurred while scrapping bus details for https://www.redbus.in/bus-tickets/mopa-airport-to-calangute-goa: Message: 
Stacktrace:
0   chromedriver                        0x0000000104805208 cxxbridge1$str$ptr + 1927396
1   chromedriver                        0x00000001047fd66c cxxbridge1$str$ptr + 1895752
2   chromedriver                        0x00000001043f8808 cxxbridge1$string$len + 89564
3   chromedriver                        0x000000010443cbcc cxxbridge1$string$len + 369056
4   chromedriver                        0x0000000104476228 cxxbridge1$string$len + 604156
5   chromedriver                        0x0000000104431698 cxxbridge1$string$len + 322668
6   chromedriver                        0x0000000104432310 cxxbridge1$string$len + 325860
7   chromedriver                        0x00000001047cbe78 cxxbridge1$str$ptr + 1693012
8   chromedriver                        0x00000001047d077c cxxbridge1$str$ptr + 1711704
9   chromedriver                        0x00000001047b13ec cxxbri

In [2]:
df_buses_6

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.7,INR 1000,7 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.6,INR 660,10 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,,,,,,,,
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,,,,,,,,
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
146,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 31m,16:01,3.7,INR 100,41 Seats available
147,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,UNIQUE TOURS AND TRAVELS,A/C Seater Push Back (2+1),08:30,11h 30m,20:00,3.2,INR 400,17 Seats available
148,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Humsafar Travels,Bharat Benz A/C Sleeper (2+1),03:30,04h 30m,08:00,1.3,INR 1350,9 Seats available
149,Belagavi to Goa,https://www.redbus.in/bus-tickets/belagavi-to-goa,Shirdi Xpress,A/C Sleeper (2+1),08:15,03h 15m,11:30,1.7,419,19 Seats available
